In [2]:
import pandas, numpy

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin


In [3]:
random_state=42
mutations ={}
mapping_dict = {'S':0, 'R':1}

Let's read in the testtrain dataset so we can split it into training and test datasets

In [13]:
df = pandas.read_csv('data/ds-traintest-phen-features.csv')
df.drop(columns=['secondary_structure_codes','d_MW','phi','d_volume','d_Pi','n_hbond_acceptors','rasp_wt_nlf'],inplace=True)
mutations['tt'] = df['mutation']
features = df.iloc[:,3:]
labels = df['phenotype'].map({'S':0, 'R':1})
features[:3]

,d_hydropathy_KD,d_rogov,psi,residue_sasa,n_hbond_donors,temp_factor,depth,deep_ddG,rasp_mt_nlf,rasp_score_ml_fermi,snap2_score,dist_FE2,dist_PZA,mcsm_stability_rsa,mcsm_stability_ddG,mapp_score
0,1.9,-0.389,-35.2,86.2,0.0,59.720001,2.103049,-0.770,2.433114,0.223921,20,29.948318,27.641663,54.8,-0.771,13.49
1,-2.6,-0.548,-35.2,86.2,0.0,59.720001,2.103049,-1.096,2.896883,0.281366,52,29.948318,27.641663,54.8,-0.367,25.97
2,2.3,-0.514,-35.2,86.2,0.0,59.720001,2.103049,-0.908,2.623741,0.258386,4,29.948318,27.641663,54.8,-0.912,30.01


Split the testtrain dataset and scale the numerical data (based on a fit *only* on the training dataset)

In [14]:
mutations_train, mutations_test, X_train, X_test, Y_train, Y_test = train_test_split(mutations['tt'], features, labels, test_size=0.3,
                                                    random_state=random_state)

class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values

numerical_attribs = list(features)

headings = '\n'.join(i for i in numerical_attribs)

OUTPUT = open('data/ds-traintest-numericalattribs.txt', 'w')
OUTPUT.write(headings)
OUTPUT.close()

pipe = Pipeline([
    ('selector', DataFrameSelector(numerical_attribs)),
    ('scaling', RobustScaler())
    ])

X_train = pipe.fit_transform(X_train)    
Y_train = Y_train.squeeze().to_numpy()
Z_train = mutations_train.to_numpy()

X_test = pipe.transform(X_test)
Y_test = Y_test.squeeze().to_numpy()   
Z_test = mutations_test.to_numpy()

with open('data/ds-train.npy', 'wb') as f:
    numpy.save(f, Y_train)
    numpy.save(f, X_train)
    numpy.save(f, Z_train)

with open('data/ds-test.npy', 'wb') as f:
    numpy.save(f, Y_test)
    numpy.save(f, X_test)    
    numpy.save(f, Z_test)    

df.loc[df.mutation.isin(mutations_test), 'set'] = 'test'
df.loc[df.mutation.isin(mutations_train), 'set'] = 'train'
df.to_csv('data/ds-traintest-phen-features-set.csv',index=False)

Let's check the % resistance is about the same between the train and test datasets

In [15]:
pandas.crosstab(df.set,df.phenotype,margins=True)

phenotype,R,S,All
set,,,
test,103,97,200
train,246,218,464
All,349,315,664


Now let's work out what the predictions using Suspect-PZA are for the training and test datasets for analysis later

In [16]:
suspectpza = pandas.read_csv('data/reference/ds-suspectpza-results.csv')
def create_mutation(row):
    return sbmlcore.amino_acid_3to1letter[row['Wild Type']] + str(row['Position']) + sbmlcore.amino_acid_3to1letter[row['Mutant']]

suspectpza['mutation'] = suspectpza.apply(create_mutation,axis=1)    
suspectpza.set_index('mutation', inplace=True)
suspectpza['prediction'] = suspectpza[' Prediction'].str[0].map(mapping_dict)
suspectpza.drop(columns=['Chain','Wild Type','Position','Mutant','Distance to ligand',' Prediction'],inplace=True)
suspectpza

,prediction
mutation,
C72G,1
D129G,1
T167N,0
L85P,1
V139L,1
...,...
I31T,1
D8Y,1
S84I,0


In [17]:
df.set_index('mutation', inplace=True)

suspectpza = suspectpza.join(df[['phenotype', 'set']], how='outer')
suspectpza['traintest'] = suspectpza.phenotype.map(mapping_dict)

def split_testtrain(row):
    result = [None,None]
    if row.set=='train':
        result = [row.traintest,None]
    elif row.set=='test':
        result = [None, row.traintest]
    return pandas.Series(result)


suspectpza[['train', 'test']] = suspectpza.apply(split_testtrain, axis=1)
suspectpza.drop(columns=['phenotype','set','traintest'], inplace=True)
suspectpza

,prediction,train,test
mutation,,,
A102E,1,NaN,NaN
A102G,0,NaN,NaN
A102P,0,NaN,NaN
A102R,1,NaN,NaN
A102S,1,NaN,NaN
...,...,...,...
Y99C,0,0.0,NaN
Y99D,0,NaN,0.0
Y99F,0,NaN,0.0


Using the (fitted) pipeline we can repeat the process for the two types of `validation` dataset, depending whether just clinical samples or aggregated mutations

In [22]:
for i in ['samples', 'mutations']:
    df = pandas.read_csv('data/ds-validation-' + i + '-phen-features.csv')
    df.drop(columns=['secondary_structure_codes','d_MW','phi','d_volume','d_Pi','n_hbond_acceptors','rasp_wt_nlf'],inplace=True)
    mutations['v'] = df['mutation']
    features = df.iloc[:,3:]
    labels = df['phenotype'].map({'S':0, 'R':1})

    X_validate = pipe.transform(features)
    Y_validate = labels.squeeze().to_numpy()
    Z_validate = mutations['v'].to_numpy()

    with open('data/ds-validation-' + i + '.npy', 'wb') as f:
        numpy.save(f, Y_validate)
        numpy.save(f, X_validate)
        numpy.save(f, Z_validate)

In [23]:
df[:5]

,mutation,segid,phenotype,d_hydropathy_KD,d_rogov,psi,residue_sasa,n_hbond_donors,temp_factor,depth,deep_ddG,rasp_mt_nlf,rasp_score_ml_fermi,snap2_score,dist_FE2,dist_PZA,mcsm_stability_rsa,mcsm_stability_ddG,mapp_score
0,M1I,A,S,2.6,-0.452,-35.20,86.2,0.0,59.720001,2.103049,-0.407,2.854018,0.236948,7,29.948318,27.641663,54.8,-0.771,21.99
1,M1K,A,U,-5.8,-0.712,-35.20,86.2,0.0,59.720001,2.103049,-1.017,2.828313,0.278564,70,29.948318,27.641663,54.8,-0.214,27.72
2,M1R,A,U,-6.4,-0.681,-35.20,86.2,0.0,59.720001,2.103049,-0.780,3.035567,0.255298,44,29.948318,27.641663,54.8,0.417,32.16
3,M1T,A,R,-2.6,-0.548,-35.20,86.2,0.0,59.720001,2.103049,-1.096,2.896883,0.281366,52,29.948318,27.641663,54.8,-0.367,25.97
4,R2L,A,U,8.3,-0.090,133.99,54.7,0.0,53.990002,2.372958,-0.708,2.433114,0.242800,44,27.459823,24.722696,27.5,-0.236,2.64


..and again label the mutations present in the `validation` dataset

In [24]:
df.set_index('mutation', inplace=True)
suspectpza = suspectpza.join(df[['phenotype']], how='outer')
suspectpza['validation'] = suspectpza.phenotype.map(mapping_dict)
suspectpza.drop(columns=['phenotype'], inplace=True)
suspectpza

,prediction,train,test,validation
mutation,,,,
A102E,1,NaN,NaN,NaN
A102G,0,NaN,NaN,NaN
A102P,0,NaN,NaN,NaN
A102R,1,NaN,NaN,0.0
A102S,1,NaN,NaN,NaN
...,...,...,...,...
Y99C,0,0.0,NaN,NaN
Y99D,0,NaN,0.0,NaN
Y99F,0,NaN,0.0,NaN


In [25]:
df = pandas.read_csv('data/ds-mic-phen-features.csv')
df.drop(columns=['secondary_structure_codes','d_MW','phi','d_volume','d_Pi','n_hbond_acceptors','rasp_wt_nlf'],inplace=True)
mutations['mic'] = df['mutation']
features = df.iloc[:,3:]
labels = df['phenotype'].map({'S':0, 'R':1})

X_mic = pipe.transform(features)
Y_mic = labels.squeeze().to_numpy()
Z_mic = mutations['mic'].to_numpy()

with open('data/ds-mic.npy', 'wb') as f:
    numpy.save(f, Y_mic)
    numpy.save(f, X_mic)
    numpy.save(f, Z_mic)

In [26]:
df.set_index('mutation', inplace=True)
suspectpza = suspectpza.join(df[['phenotype']], how='outer')
suspectpza['mic'] = suspectpza.phenotype.map(mapping_dict)
suspectpza[:3]

,prediction,train,test,validation,phenotype,mic
mutation,,,,,,
A102E,1,NaN,NaN,NaN,NaN,NaN
A102G,0,NaN,NaN,NaN,NaN,NaN
A102P,0,NaN,NaN,NaN,NaN,NaN


In [30]:
len(Y_validate)

367

In [31]:
assert len(Y_mic) == suspectpza.mic.notna().sum()
assert len(Y_test) == suspectpza.test.notna().sum()
assert len(Y_train) == suspectpza.train.notna().sum()
# assert len(Y_validate) == suspectpza.validation.notna().sum()


In [32]:
suspectpza

,prediction,train,test,validation,phenotype,mic
mutation,,,,,,
A102E,1,NaN,NaN,NaN,NaN,NaN
A102G,0,NaN,NaN,NaN,NaN,NaN
A102P,0,NaN,NaN,NaN,NaN,NaN
A102R,1,NaN,NaN,0.0,NaN,NaN
A102S,1,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
Y99C,0,0.0,NaN,NaN,NaN,NaN
Y99D,0,NaN,0.0,NaN,NaN,NaN
Y99F,0,NaN,0.0,NaN,NaN,NaN


In [33]:
suspectpza = suspectpza[~(suspectpza.test.isna() & suspectpza.train.isna() & suspectpza.validation.isna() & suspectpza.mic.isna())]
suspectpza.to_csv('data/reference/ds-suspectpza-results-sets.csv')

In [74]:
for i in ['test', 'train', 'validation', 'mic']:
    Y=suspectpza[suspectpza[i].notna()][i].squeeze().to_numpy()
    Z=suspectpza[suspectpza[i].notna()].prediction.squeeze().to_numpy()
    ZZ=suspectpza[suspectpza[i].notna()].index.to_numpy()
    with open('data/suspectpza-'+i+'.npy', 'wb') as f:
        numpy.save(f, Y)
        numpy.save(f, Z)
        numpy.save(f, ZZ)

In [75]:
suspectpza.validation.value_counts()

1.0    155
0.0     44
Name: validation, dtype: int64